In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.model_selection import train_test_split


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [ ]:
img_size = (224, 224)

data_dir = "C:\\Users\\batuy\\Documents\\VS Code\\drugs_classification\\Data Combined"
train_dir = "train_data"
test_dir = "test_data"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)


In [ ]:
# Her bir kategori için veri ayırma işlemi
for category in os.listdir(data_dir):
    category_path = os.path.join(data_dir, category)
    if os.path.isdir(category_path):
        # Kategoriye ait tüm dosyaları listeleme
        files = os.listdir(category_path)
        train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
        
        # Eğitim ve test dizinlerine bu kategori için alt dizinler oluşturma
        train_category_dir = os.path.join(train_dir, category)
        test_category_dir = os.path.join(test_dir, category)
        
        os.makedirs(train_category_dir, exist_ok=True)
        os.makedirs(test_category_dir, exist_ok=True)
        
        # Dosyaları ilgili dizinlere taşıma
        for file in train_files:
            shutil.copy(os.path.join(category_path, file), os.path.join(train_category_dir, file))
        
        for file in test_files:
            shutil.copy(os.path.join(category_path, file), os.path.join(test_category_dir, file))


In [ ]:
train_data_gen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=40,
                                   horizontal_flip=True)

test_data_gen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_data = train_data_gen.flow_from_directory(directory=train_dir,
                                               batch_size=64,
                                               class_mode="categorical",
                                               target_size=img_size)

test_data = test_data_gen.flow_from_directory(directory=test_dir,
                                              batch_size=64,
                                              class_mode="categorical",
                                              target_size=img_size)

base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

x = base_model.output
x = Flatten()(x)
x = Dense(350, activation="relu")(x)
prediction = Dense(10, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=prediction)


In [ ]:
import random
import os
import matplotlib.pyplot as plt

# Ana veri dizini
target_1 = "C:\\Users\\batuy\\Documents\\VS Code\\drugs_classification\\train_data"

target_list = os.listdir(target_1)

random_path = random.choice(target_list)

klasor_yolu = os.path.join(target_1, random_path)

dosya_listesi = os.listdir(klasor_yolu)

resimler = [resim for resim in dosya_listesi if resim.endswith(".png") or resim.endswith(".jpg")]

random_img = random.sample(resimler, 6)

for resim in random_img:
    print("Path: ", os.path.join(klasor_yolu, resim)[62:74])
    resim_yolu = os.path.join(klasor_yolu, resim)
    img = plt.imread(resim_yolu)
    print(f"Shape: ", {img.shape})
    plt.imshow(img)
    plt.title(resim)   
    plt.show()

In [ ]:
base_model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

rlp = ReduceLROnPlateau(monitor='val_accuracy', factor=0.3, min_lr=0.000001, patience=2)
# model_check = ModelCheckpoint(filepath="project.keras", monitor="val_loss", verbose=1 ,save_best_only=True)
callbacks = [rlp]

# train_data_2, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [ ]:
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=30,
                    batch_size=64,
                    callbacks=callbacks)

In [ ]:
model.evaluate()

In [ ]:
plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.legend()
plt.show()